In [ ]:
#! curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#! python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev
#import torch_xla
# imports pytorch
#import torch

# imports the torch_xla package
#import torch_xla
#import torch_xla.core.xla_model as xm

#device = xm.xla_device()
#device

In [ ]:
!pip install pydub

In [ ]:
import numpy as np 
import pandas as pd 
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [ ]:
def count_csv_files(root_folder):
    csv_count = 0

    # Loop over all folders and files in the root_folder
    for folder_name, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                csv_count += 1

    return csv_count

# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Annotations'
csv_count = count_csv_files(root_folder)

if csv_count > 0:
    print(f"Number of .csv files found: {csv_count}")
else:
    print("No .csv files found.")


In [ ]:
def insert_audio_clip(background, audio_clip):
    """
    Insert a new audio segment over the background noise at a random time step, ensuring that the
    audio segment does not overlap with existing segments.

    Arguments:
    background -- a 10 second background audio recording.
    audio_clip -- the audio clip to be inserted/overlaid.
    previous_segments -- times where audio segments have already been placed

    Returns:
    new_background -- the updated background audio
    """
    background -= 25
    # Get the duration of the audio clip in ms
    segment_ms = len(audio_clip)


    new_background = background.overlay(audio_clip, position = 0)

    return new_background

In [ ]:
Out_Noise_data = []

In [ ]:
from pydub import AudioSegment
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
def find_min_speech_length_and_path(root_folder,Out_Noise_data):
    min_length = float('inf')  # Set initial value to positive infinity
    min_path = None
    
    # Loop over all folders and files in the root_folder
    for folder_name, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:

            if filename.lower().endswith('.wav'):
                file_path = os.path.join(folder_name, filename)
                audio = AudioSegment.from_wav(file_path)
                
                Out_Noise_data.append(audio)

                # Open the wav file using sf
                with sf.SoundFile(file_path, 'r') as file:
                    # Get the duration in seconds
                    duration_seconds = len(file) / file.samplerate

                    # Convert to milliseconds
                    duration_ms = int(duration_seconds * 1000)

                    # Update min_length and min_path if the current speech length is smaller
                    if duration_ms < min_length:
                        min_length = duration_ms
                        min_path = file_path

    return Out_Noise_data, min_length, min_path

# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/noise-data-set/noise_train'
Out_Noise_data,min_speech_length, min_path = find_min_speech_length_and_path(root_folder,Out_Noise_data)

if min_path:
    print(f"The minimum speech length is {min_speech_length} milliseconds, found in file: {min_path}")
else:
    print("No .wav files found.")


In [ ]:
len(Out_Noise_data)

In [ ]:
import os
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
def find_min_speech_length_and_path(root_folder,Out_Noise_data):
    min_length = float('inf')  # Set initial value to positive infinity
    min_path = None
    
    # Loop over all folders and files in the root_folder
    for folder_name, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:

            if filename.lower().endswith('.wav'):
                file_path = os.path.join(folder_name, filename)
                audio = AudioSegment.from_wav(file_path)
                Out_Noise_data.append(audio)

                # Open the wav file using sf
                with sf.SoundFile(file_path, 'r') as file:
                    # Get the duration in seconds
                    duration_seconds = len(file) / file.samplerate

                    # Convert to milliseconds
                    duration_ms = int(duration_seconds * 1000)

                    # Update min_length and min_path if the current speech length is smaller
                    if duration_ms < min_length:
                        min_length = duration_ms
                        min_path = file_path

    return Out_Noise_data, min_length, min_path

# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/noise-data-set/noise_test'
Out_Noise_data,min_speech_length, min_path = find_min_speech_length_and_path(root_folder,Out_Noise_data)

if min_path:
    print(f"The minimum speech length is {min_speech_length} milliseconds, found in file: {min_path}")
else:
    print("No .wav files found.")


In [ ]:
len(Out_Noise_data)

In [ ]:
import os
import soundfile as sf
import random
import librosa
import numpy as np
AudioSegment.converter = "/path/to/ffmpeg"
def find_max_speech_length(root_folder, Destination, Out_Noise_data):
    max_length = 0

    # Loop over all folders and files in the root_folder
    for folder_name, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith('.flac'):
                file_path = os.path.join(folder_name, filename)

                # Replace 'output.wav' with the desired path and name for the output WAV file
                output_file = "/kaggle/working/sample.wav"
                # Open the FLAC file and convert it to WAV
                # Read the FLAC audio file             
                data, current_sample_rate = sf.read(file_path)
                 
                #break
                sf.write(output_file, data, current_sample_rate)

                AudioSegment.converter = "/path/to/ffmpeg"
                Audio = AudioSegment.from_wav(output_file)
                Noise =  Out_Noise_data[random.randrange(0, 179)]
                audio_clip= None
                if(len(Noise) > len(Audio)):
                    audio_clip = insert_audio_clip(Noise[:len(Audio)], Audio)
                else:
                    repeat_factor = int(np.ceil((len(Audio)+0.0) / len(Noise)))
                    # Repeat the noise
                    RepeatedNoise = Noise * repeat_factor
                    # Trim the repeated noise to match the length of the audio
                    audio_clip = insert_audio_clip(RepeatedNoise[:len(Audio)], Audio)

                output_path = os.path.join(Destination, file_path[40:-5] + '.wav')
                os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Create directories if they don't exist
                audio_clip.export(output_path, format="wav")



    return max_length

# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Audio'
Destination = '/kaggle/working/LibriStutter_Data/'
max_speech_length = find_max_speech_length(root_folder, Destination, Out_Noise_data)


print(f"The maximum speech length is {max_speech_length} milliseconds.")


In [ ]:
!pip install tiktoken
!pip install more_itertools

In [ ]:
import os
from functools import lru_cache
from subprocess import CalledProcessError, run
from typing import Optional, Union
import numpy as np
import torch
import torch.nn.functional as F
import os
os.chdir("/kaggle/input/feature-extraction")
from audio import (FRAMES_PER_SECOND,HOP_LENGTH,
    N_FRAMES,N_SAMPLES,SAMPLE_RATE, CHUNK_LENGTH, FRAMES_PER_SECOND,
    load_audio, exact_div, pad_or_trim, mel_filters, log_mel_spectrogram)
os.chdir("/kaggle/input/filesa-data-speech/")
from english import EnglishTextNormalizer
from english import (EnglishNumberNormalizer,EnglishSpellingNormalizer,)
from basic import BasicTextNormalizer as BasicTextNormalizer
#!pip install  ffmpeg

In [ ]:
Norm11=BasicTextNormalizer()
Norm12=EnglishTextNormalizer()
Norm13=EnglishSpellingNormalizer()
Norm14=EnglishNumberNormalizer()

In [ ]:
%%capture
!apt update -y && apt upgrade -y
!apt install sox libsox-fmt-mp3 -y
!apt install ffmpeg -y

In [ ]:
'''import os
import pandas as pd
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
L_Audio=[]
L_Txt=[]

def find_min_speech_length_and_path(root_folder):
    # Loop over all folders and files in the root_folder
    io = 0
    exi = False
    for folder_name, subfolders, filenames in os.walk(root_folder):
        print(folder_name)
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                file_path = '/kaggle/working/LibriStutter_Data/ Audio'+folder_name[52:]+'/'+filename[:-3]+'wav'
                io+=1
                #print(file_path)
                L_Audio.append(pad_or_trim(log_mel_spectrogram(file_path, 80, padding=N_SAMPLES), N_FRAMES))
                file_path = os.path.join(folder_name, filename)
                df = pd.read_csv(file_path,header =None)
                formatted_line=""
                for _, row in df.iterrows():
                    text = row[0]
                    start_time = row[1]
                    end_time = row[2]
                    
                    if start_time < 0 or start_time > 30 or round(start_time * 100) % 2 != 0 or end_time < 0 or end_time > 30 or round(end_time * 100) % 2 != 0:
                        formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                    else:
                        formatted_line += f"<|{start_time:.2f}|> {Norm14(Norm13(Norm12(Norm11(text))))}<|{end_time:.2f}|>"
                    #formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                #with open(file_path, 'r') as file:
                #    file_contents = file.read()
                L_Txt.append(formatted_line)
                if(io == 1000):
                    exi = True
                    break
        if(exi):
            break
    
# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Annotations'
find_min_speech_length_and_path(root_folder)

'''

In [ ]:
'''import os
import pandas as pd
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
L_Audio=[]
L_Txt=[]

def find_min_speech_length_and_path(root_folder):
    # Loop over all folders and files in the root_folder
    io = 0
    
    exi = False
    for folder_name, subfolders, filenames in os.walk(root_folder):
        print(folder_name)
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                file_path = '/kaggle/working/LibriStutter_Data/ Audio'+folder_name[52:]+'/'+filename[:-3]+'wav'
                io+=1
                if(io > 1000 ):
                    #print(file_path)
                    L_Audio.append(pad_or_trim(log_mel_spectrogram(file_path, 80, padding=N_SAMPLES), N_FRAMES))
                    file_path = os.path.join(folder_name, filename)
                    df = pd.read_csv(file_path,header =None)
                    formatted_line=""
                    for _, row in df.iterrows():
                        text = row[0]
                        start_time = row[1]
                        end_time = row[2]

                        if start_time < 0 or start_time > 30 or round(start_time * 100) % 2 != 0 or end_time < 0 or end_time > 30 or round(end_time * 100) % 2 != 0:
                            formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                        else:
                            formatted_line += f"<|{start_time:.2f}|> {Norm14(Norm13(Norm12(Norm11(text))))}<|{end_time:.2f}|>"
                        #formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                    #with open(file_path, 'r') as file:
                    #    file_contents = file.read()
                    L_Txt.append(formatted_line)
                if(io == 2000):
                    exi = True
                    break
        if(exi):
            break
    
# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Annotations'
find_min_speech_length_and_path(root_folder)
'''

In [ ]:
'''import os
import pandas as pd
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
L_Audio=[]
L_Txt=[]

def find_min_speech_length_and_path(root_folder):
    # Loop over all folders and files in the root_folder
    io = 0
    
    exi = False
    for folder_name, subfolders, filenames in os.walk(root_folder):
        print(folder_name)
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                file_path = '/kaggle/working/LibriStutter_Data/ Audio'+folder_name[52:]+'/'+filename[:-3]+'wav'
                io+=1
                if(io > 2000 ):
                    #print(file_path)
                    L_Audio.append(pad_or_trim(log_mel_spectrogram(file_path, 80, padding=N_SAMPLES), N_FRAMES))
                    file_path = os.path.join(folder_name, filename)
                    df = pd.read_csv(file_path,header =None)
                    formatted_line=""
                    for _, row in df.iterrows():
                        text = row[0]
                        start_time = row[1]
                        end_time = row[2]

                        if start_time < 0 or start_time > 30 or round(start_time * 100) % 2 != 0 or end_time < 0 or end_time > 30 or round(end_time * 100) % 2 != 0:
                            formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                        else:
                            formatted_line += f"<|{start_time:.2f}|> {Norm14(Norm13(Norm12(Norm11(text))))}<|{end_time:.2f}|>"
                        #formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                    #with open(file_path, 'r') as file:
                    #    file_contents = file.read()
                    L_Txt.append(formatted_line)
                if(io == 3000):
                    exi = True
                    break
        if(exi):
            break
    
# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Annotations'
find_min_speech_length_and_path(root_folder)

'''

In [ ]:
import os
import pandas as pd
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
L_Audio=[]
L_Txt=[]

def find_min_speech_length_and_path(root_folder):
    # Loop over all folders and files in the root_folder
    io = 0
    
    exi = False
    for folder_name, subfolders, filenames in os.walk(root_folder):
        print(folder_name)
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                file_path = '/kaggle/working/LibriStutter_Data/ Audio'+folder_name[52:]+'/'+filename[:-3]+'wav'
                io+=1
                if(io > 3000 ):
                    #print(file_path)
                    L_Audio.append(pad_or_trim(log_mel_spectrogram(file_path, 80, padding=N_SAMPLES), N_FRAMES))
                    file_path = os.path.join(folder_name, filename)
                    df = pd.read_csv(file_path,header =None)
                    formatted_line=""
                    for _, row in df.iterrows():
                        text = row[0]
                        start_time = row[1]
                        end_time = row[2]

                        if start_time < 0 or start_time > 30 or round(start_time * 100) % 2 != 0 or end_time < 0 or end_time > 30 or round(end_time * 100) % 2 != 0:
                            formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                        else:
                            formatted_line += f"<|{start_time:.2f}|> {Norm14(Norm13(Norm12(Norm11(text))))}<|{end_time:.2f}|>"
                        #formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                    #with open(file_path, 'r') as file:
                    #    file_contents = file.read()
                    L_Txt.append(formatted_line)
                if(io == 4000):
                    exi = True
                    break
        if(exi):
            break
    
# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Annotations'
find_min_speech_length_and_path(root_folder)



In [ ]:
import os
import pandas as pd
import soundfile as sf
AudioSegment.converter = "/path/to/ffmpeg"
L_Audio_val=[]
L_Txt_val=[]

def find_min_speech_length_and_path(root_folder):
    # Loop over all folders and files in the root_folder
    io = 0
    
    exi = False
    for folder_name, subfolders, filenames in os.walk(root_folder):
        print(folder_name)
        for filename in filenames:
            if filename.lower().endswith('.csv'):
                file_path = '/kaggle/working/LibriStutter_Data/ Audio'+folder_name[52:]+'/'+filename[:-3]+'wav'
                io+=1
                if(io > 4000 ):
                    #print(file_path)
                    L_Audio_val.append(pad_or_trim(log_mel_spectrogram(file_path, 80, padding=N_SAMPLES), N_FRAMES))
                    file_path = os.path.join(folder_name, filename)
                    df = pd.read_csv(file_path,header =None)
                    formatted_line=""
                    for _, row in df.iterrows():
                        text = row[0]
                        start_time = row[1]
                        end_time = row[2]

                        if start_time < 0 or start_time > 30 or round(start_time * 100) % 2 != 0 or end_time < 0 or end_time > 30 or round(end_time * 100) % 2 != 0:
                            formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                        else:
                            formatted_line += f"<|{start_time:.2f}|> {Norm14(Norm13(Norm12(Norm11(text))))}<|{end_time:.2f}|>"
                        #formatted_line +=Norm14(Norm13(Norm12(Norm11(text))))
                    #with open(file_path, 'r') as file:
                    #    file_contents = file.read()
                    L_Txt_val.append(formatted_line)
                if(io == 4500):
                    exi = True
                    break
        if(exi):
            break
    
# Replace '/path/to/root/folder' with the actual path to your root folder
root_folder = '/kaggle/input/libri-stutter/LibriStutter Annotations'
find_min_speech_length_and_path(root_folder)



In [ ]:
print('len(L_Audio) -> ', len(L_Audio))
print('len(L_Audio_val) -> ',len(L_Audio_val))

In [ ]:
def exact_div(x, y):
    assert x % y == 0
    return x // y


# hard-coded audio hyperparameters
SAMPLE_RATE = 16000
N_FFT = 400
HOP_LENGTH = 160
CHUNK_LENGTH = 30 # 30-second
N_SAMPLES = CHUNK_LENGTH * SAMPLE_RATE  # 480000 samples in a 30-second chunk
N_FRAMES = exact_div(N_SAMPLES, HOP_LENGTH)  # 3000 frames in a mel spectrogram input
N_SAMPLES_PER_TOKEN = HOP_LENGTH * 2  # the initial convolutions has stride 2
FRAMES_PER_SECOND = exact_div(SAMPLE_RATE, HOP_LENGTH)  # 10ms per audio frame
TOKENS_PER_SECOND = exact_div(SAMPLE_RATE, N_SAMPLES_PER_TOKEN)  # 20ms per audio token

In [ ]:
!pip install tiktoken
!pip install more_itertools


In [ ]:
import os
os.chdir("/kaggle/input/tokenizer")
from tokenizer import Tokenizer, get_tokenizer
os.chdir("/kaggle/input")
tokenizer =  get_tokenizer()
from torch.utils.data import DataLoader, Dataset
from typing import  List, Optional, Tuple, Union
import re

In [ ]:
tokenizer.eot

In [ ]:
class AudioDataset(Dataset):
    def __init__(
        self,
        mels,
        records,
        tokenizer: Tokenizer,
        fp16: bool = True,
        no_timestamps_training: bool = False,
        max_prompt_length: int = 223,  # The maximum number of tokens to use for the prompt
        prompt_use_rate: float = 0.5,
        no_timestamps_rate: float = 0.5,
    ) -> None:
        self.mels = mels
        self.records = records
        self.tokenizer = tokenizer
        self.fp16 = fp16
        self.no_timestamps_training = no_timestamps_training
        self.max_prompt_length = max_prompt_length
        self.prompt_use_rate = prompt_use_rate
        self.no_timestamps_rate = no_timestamps_rate

        self.num_frames_per_second = N_FRAMES / CHUNK_LENGTH
        # timestamps tokens are from <|0.00|> to <|30.00|> with a step of 0.02
        self.timestamp_pattern = re.compile(r"(<\|[123]?[0-9]\.[0-9][0-9]\|>)")
        self.model_n_text_ctx = 448

    def __len__(self) -> int:
        return len(self.records)

    def _get_prompt_tokens(self, prompt: str) -> List[int]:
        if len(prompt) > 0 and torch.rand(1) < self.prompt_use_rate:
            prompt_tokens = self._encode_text_with_timestamps(prompt)[-self.max_prompt_length :]
            prompt_tokens = [self.tokenizer.sot_prev] + prompt_tokens
        else:
            prompt_tokens = []

        return prompt_tokens

    def _get_special_tokens(
        self, is_text_empty: bool, language: str, no_timestamps: bool
    ) -> List[int]:
        if is_text_empty:
            special_tokens = [self.tokenizer.sot, self.tokenizer.no_speech]
        else:
            special_tokens = [
                self.tokenizer.sot,
                self.tokenizer.special_tokens[f"<|{language}|>"],
                self.tokenizer.special_tokens["<|transcribe|>"],
            ]
            if no_timestamps:
                special_tokens.append(self.tokenizer.no_timestamps)

        return special_tokens

    def _encode_text_with_timestamps(self, text: str) -> List[int]:
        parts = self.timestamp_pattern.split(text)
        parts = [token for token in parts if token != ""]
        tokens = []
        for part in parts:
            if self.timestamp_pattern.fullmatch(part) is not None:
                timestamp = float(part[2:-2])

                # timestamp must be in the range [0, 30] and be a multiple of 0.02 seconds
                if timestamp < 0 or timestamp > 30 or round(timestamp * 100) % 2 != 0:
                    raise ValueError(f"Invalid timestamp: {timestamp}")

                token = self.tokenizer.timestamp_begin + round(timestamp * 100) // 2
                tokens.append(token)
            else:
                tokens.extend(self.tokenizer.encode(part))

        return tokens

    def _get_partial_segment_start(self, tokens: List[int]) -> Optional[float]:
        if (
            len(tokens) >= 2
            and tokens[-2] >= self.tokenizer.timestamp_begin
            and tokens[-1] >= self.tokenizer.timestamp_begin
        ):  # if the last token is a start time token
            return (tokens[-1] - self.tokenizer.timestamp_begin) * 0.02
        else:
            return None

    def _get_text_tokens(self, text: str, no_timestamps: bool) -> Tuple[List[int], Optional[float]]:
        text_tokens = self._encode_text_with_timestamps(text)
        next_partial_segment_start = self._get_partial_segment_start(text_tokens)
        if no_timestamps:
            text_tokens = list(filter(lambda x: x < self.tokenizer.timestamp_begin, text_tokens))

        return text_tokens, next_partial_segment_start

    def _calculate_mel(
        self, audio_path: str, next_partial_segment_start: Optional[float], no_timestamps: bool
    ) -> torch.Tensor:
        mel = log_mel_spectrogram(audio_path)
        if no_timestamps and next_partial_segment_start is not None:
            mel = mel[:, : int(next_partial_segment_start * self.num_frames_per_second)]
        mel = pad_or_trim(mel, N_FRAMES)
        if self.fp16:
            mel = mel.half()

        return mel

    def _construct_decoder_output(
        self, prompt_tokens: List[int], special_tokens: List[int], text_tokens: List[int]
    ) -> List[int]:
        if len(prompt_tokens) == 0:
            decoder_output = special_tokens[1:] + text_tokens + [self.tokenizer.eot]
        else:
            decoder_output = (
                # Mask out the training loss for predicting the prompt tokens. We use "-100" as the
                # default value for the `ignore_index` parameter in
                # `torch.nn.functional.cross_entropy()`. However, we do not mask out the loss for
                # predicting the sot token because our experiment indicates that the original
                # Whisper model assigns a high probability to the sot token after prompt tokens.
                [-100] * (len(prompt_tokens) - 1)
                + special_tokens
                + text_tokens
                + [self.tokenizer.eot]
            )
        return decoder_output

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        record = self.records[index]
        mel = self.mels[index]
        no_timestamps = self.no_timestamps_training
        #no_timestamps = True
        #print(no_timestamps)
        text_tokens, next_partial_segment_start = self._get_text_tokens(record, no_timestamps)
        is_text_empty = len(text_tokens) == 0
        special_tokens = self._get_special_tokens(is_text_empty, "en", no_timestamps)

        decoder_input = special_tokens + text_tokens
        if len(decoder_input) > self.model_n_text_ctx:
            raise ValueError(f"Input is too long: {record} (length: {len(decoder_input)})")

        decoder_output = self._construct_decoder_output([],special_tokens, text_tokens)

        
        return (
            mel,
            torch.tensor(decoder_input, dtype=torch.long),
            torch.tensor(decoder_output, dtype=torch.long),
        )

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def collate_fn(data):
    x, y_in, y_out = zip(*data)
    x = pad_sequence(x, batch_first=True, padding_value=0)
    y_in = pad_sequence(y_in, batch_first=True, padding_value=50256)#eot token
    y_out = pad_sequence(y_out, batch_first=True, padding_value=-100)
    return x, y_in, y_out
tokenizer =  get_tokenizer()
dataset = AudioDataset(
    L_Audio,
    L_Txt,
    tokenizer,
    no_timestamps_training=False)
DataLoaders = DataLoader(
        dataset,
        batch_size=1,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        collate_fn=collate_fn,
    )

dataset_val = AudioDataset(
    L_Audio_val,
    L_Txt_val,
    tokenizer,
    no_timestamps_training=False)
DataLoaders_val = DataLoader(
        dataset_val,
        batch_size=4,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        collate_fn=collate_fn,
    )

In [ ]:

import os
os.chdir("/kaggle/input/whispersa")
from model import (Whispersa,TextDecoder,AudioEncoder,ModelDimensions)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #"cpu"

In [ ]:
# Create an instance of ModelDimensions
dimensions_instance = ModelDimensions(
    n_mels=80, n_audio_ctx=1500, n_audio_state=768, n_audio_head=12, n_audio_layer=12, n_vocab=tokenizer.encoding.n_vocab, n_text_ctx=448, n_text_state=768, n_text_head=12, n_text_layer=12
   # n_mels=80, n_audio_ctx=1500, n_audio_state=768, n_audio_head=12, n_audio_layer=12, n_vocab=51864, n_text_ctx=448, n_text_state=768, n_text_head=12, n_text_layer=12
  
)

In [ ]:
'''
!pip install git+https://github.com/openai/whisper.git
import whisper
modela = whisper.load_model('small.en').to(device)
model = Whispersa(dimensions_instance).to(device)
model.load_state_dict(modela.state_dict())
'''

In [ ]:

import torch
model = Whispersa(dimensions_instance).to(device)
save_path = "/kaggle/input/speech-pro1/model2.pt"  # Replace with your desired path
model.load_state_dict(torch.load(save_path))

In [ ]:
def evaluate(model, iterator, criterion,device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
         for batch in iterator:
            x, y_in, y_out = batch
            y_in,x, y_out = y_in.to(device), x.to(device), y_out.to(device)
            audio_features = model.embed_audio(x)  
            logits = model.logits(y_in, audio_features=audio_features)
            loss = loses(logits.view(-1, logits.size(-1)), y_out.view(-1))
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [ ]:
import torch.nn.functional as F

def train_step(model,train_iter,optimizer: torch.optim.Optimizer,accum_grad_steps: int,train_only_decoder: bool,loses,device):
    model.train()
    total_loss = 0
    for _ in range(accum_grad_steps):
        x, y_in, y_out = next(train_iter)
        y_in,x, y_out = y_in.to(device), x.to(device), y_out.to(device)
        if train_only_decoder:
            with torch.no_grad():
                audio_features = model.embed_audio(x)
        else:
            audio_features = model.embed_audio(x)        
        
        logits = model.logits(y_in, audio_features=audio_features)
        #loss = F.cross_entropy(logits.transpose(1, 2), y_out)
        #print(logits.shape)
        #print(logits.view(-1, logits.shape[-1]).shape)
        #print( logits.shape[-1])
        loss = loses(logits.view(-1, logits.size(-1)), y_out.view(-1))
        #print(logits.view(-1, tokenizer.encoding.n_vocab).shape)
        #loss = F.cross_entropy(logits.view(-1, tokenizer.encoding.n_vocab), y_out)
        loss = loss / accum_grad_steps
        loss.backward()
        total_loss += loss.item()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
    optimizer.step()
    optimizer.zero_grad()

    return total_loss

In [ ]:
optimizer =  torch.optim.AdamW(model.parameters(), lr=1e-5)
loses = torch.nn.CrossEntropyLoss(ignore_index=-100).to(device)
#loses = torch.nn.CrossEntropyLoss(ignore_index=-100)
def infinite_iter(data_loader: DataLoader):
    while True:
        for batch in data_loader:
            yield batch
from tqdm import tqdm
pbar = tqdm(range(1, 91))
train_iter = infinite_iter(DataLoaders)
for step in pbar:
    train_loss =train_step(model,train_iter,optimizer,64,False,loses,device)
    val_loss =evaluate(model,DataLoaders_val,loses,device)
    pbar.set_postfix({"loss": train_loss,"val loss": val_loss})
    print("step -> ", step," train_loss = ", train_loss," val_loss = ", val_loss)

In [ ]:
import shutil
def remove_path_and_contents(path):
    try:
        # Remove the entire path and its contents
        shutil.rmtree(path)
        print(f"Removed path and its contents: {path}")
    except Exception as e:
        print(f"Error: {e}")

# Replace 'your_path_here' with the actual path you want to remove
path_to_remove = '/kaggle/working/LibriStutter_Data'
remove_path_and_contents(path_to_remove)

In [ ]:
# Define the saving path
save_pasth = "/kaggle/working/model3.pt"  # Replace with your desired path

# Save the model state dictionary
torch.save(model.state_dict(), save_pasth)

print(f"Model saved successfully to: {save_pasth}")

In [ ]:

import torch
models = Whispersa(dimensions_instance).to(device)
save_path = "/kaggle/input/speech-pro1/model.pt"  # Replace with your desired path
models.load_state_dict(torch.load(save_path))
# Define the saving path
save_pasth = "/kaggle/working/model.pt"  # Replace with your desired path

# Save the model state dictionary
torch.save(models.state_dict(), save_pasth)

print(f"Model saved successfully to: {save_pasth}")

In [ ]:

import torch
modelsa = Whispersa(dimensions_instance).to(device)
save_path = "/kaggle/input/speech-pro1/model1.pt"  # Replace with your desired path
modelsa.load_state_dict(torch.load(save_path))
# Define the saving path
save_pasth = "/kaggle/working/model1.pt"  # Replace with your desired path

# Save the model state dictionary
torch.save(modelsa.state_dict(), save_pasth)

print(f"Model saved successfully to: {save_pasth}")

In [ ]:

import torch
modelsaa = Whispersa(dimensions_instance).to(device)
save_path = "/kaggle/input/speech-pro1/model2.pt"  # Replace with your desired path
modelsaa.load_state_dict(torch.load(save_path))
# Define the saving path
save_pasth = "/kaggle/working/model2.pt"  # Replace with your desired path

# Save the model state dictionary
torch.save(modelsaa.state_dict(), save_pasth)

print(f"Model saved successfully to: {save_pasth}")

In [ ]:
'''
import os
os.chdir("/kaggle/input/whispersa")
from model import (Whispersa,TextDecoder,AudioEncoder,ModelDimensions)
!pip install tiktoken
!pip install more_itertools
import os
os.chdir("/kaggle/input/tokenizer")
from tokenizer import Tokenizer, get_tokenizer
tokenizer =  get_tokenizer()
'''

In [ ]:
'''
# Create an instance of ModelDimensions
dimensions_instance = ModelDimensions(
       n_mels=80, n_audio_ctx=1500, n_audio_state=768, n_audio_head=12, n_audio_layer=12, n_vocab=tokenizer.encoding.n_vocab, n_text_ctx=448, n_text_state=768, n_text_head=12, n_text_layer=12

)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #"cpu"
model = Whispersa(dimensions_instance).to(device)
save_path = "/kaggle/input/speech-pro1/model1.pt"  # Replace with your desired path
model.load_state_dict(torch.load(save_path))
model.to(device)'''

In [ ]:
'''model.device'''

In [ ]:
'''
import os
os.chdir("/kaggle/input/transcribe")
from transcribe import transcribe'''

In [ ]:
'''
result = transcribe(model, '/kaggle/input/libri-stutter/LibriStutter Audio/445/123857/445-123857-0000.flac', language='en', temperature=1.0, word_timestamps=True)
print(result['text'])
'''